# Mathematical Optimisation - Project
This project aims at ricreating the results reported in:

[A fix-and-optimize heuristic for the Unrelated Parallel Machine Scheduling Problem
](https://www.sciencedirect.com/science/article/pii/S0305054823003684)

In [17]:
from Solver import Solver
from MPA.MPA import MPA
from Heuristic.Greedy import Greedy
from Heuristic.FixOpt import FixOpt
import random as r
import json
from IPython.display import clear_output

r.seed(10)
  
with open('credentials.txt') as f: 
    data = f.read() 
options = json.loads(data)

## Creation of the matrices
- P stores the execution times
- S stores the setup times
The matrices are implemented using dictionaries to handle the indexes not starting from 0

### Small instance presented in the paper

In [18]:
P_dict = {
            (1,1): 2,(1,2): 2,(1,3): 3,(1,4): 4,(2,1): 3,(2,2): 3,(2,3): 5,(2,4): 3,
         }

S_dict = {
            (1,1,1): 0,(1,1,2): 2,(1,1,3): 1,(1,1,4): 1,(1,2,1): 2,(1,2,2): 0,(1,2,3): 1,(1,2,4): 2,
            (1,3,1): 3,(1,3,2): 1,(1,3,3): 0,(1,3,4): 3,(1,4,1): 2,(1,4,2): 1,(1,4,3): 2,(1,4,4): 0,
            (2,1,1): 0,(2,1,2): 3,(2,1,3): 2,(2,1,4): 1,(2,2,1): 2,(2,2,2): 0,(2,2,3): 3,(2,2,4): 2,
            (2,3,1): 3,(2,3,2): 2,(2,3,3): 0,(2,3,4): 3,(2,4,1): 2,(2,4,2): 2,(2,4,3): 3,(2,4,4): 0,
         }

### Instance generator
The parameters are:
- The cardinality of the set of jobs N
- The cardinality of the set of machines M
- The minimum and the maximum execution time
- The minimum and the maximum setup time

In [19]:
def generate_instance(N_cardinality, M_cardinality,
                      min_execution_time = 1,max_execution_time = 10,
                      min_setup_time = 1,max_setup_time = 3):
    
    P = {}
    S = {}
    
    N = range(1, N_cardinality+1)
    M = range(1, M_cardinality+1)
    
    for i in M:
        for j in N:
            P[i,j] = r.randint(min_execution_time, max_execution_time)
            
    for i in M:
        for j in N:
            for k in N:
                S[i,j,k] = r.randint(min_setup_time, max_setup_time)
    
    return P,S

N_cardinality = 50
M_cardinality = 10

P_dict, S_dict = generate_instance(N_cardinality=N_cardinality,M_cardinality=M_cardinality)

The sets N,M,N0 are created again to print the results down the line

In [20]:
N = range(1, N_cardinality+1)
M = range(1, M_cardinality+1)
N0 = [i for i in N]
N0.insert(0,0)

# Exact Solution

The problem is solved exactly using the Solver class.

Then, the results are printed. Note that job 0 is a dummy job used to represent the beginning and the end of activities for each machine.

In [21]:
s = Solver(execution_times = P_dict, setup_times = S_dict)
decision_variables,completion_times,maximum_makespan,assignments = s.solve(options=options)
clear_output(wait=True)
for i in s.N:
    print(f'Job {i} completed at {completion_times[i]}s')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

Job 1 completed at 13.0s
Job 2 completed at 3.0s
Job 3 completed at 13.0s
Job 4 completed at 7.0s
Job 5 completed at 13.0s
Job 6 completed at 9.0s
Job 7 completed at 4.0s
Job 8 completed at 2.0s
Job 9 completed at 1.0s
Job 10 completed at 13.0s
Job 11 completed at 5.0s
Job 12 completed at 1.0s
Job 13 completed at 6.0s
Job 14 completed at 11.0s
Job 15 completed at 11.0s
Job 16 completed at 8.0s
Job 17 completed at 5.0s
Job 18 completed at 3.0s
Job 19 completed at 3.0s
Job 20 completed at 7.0s
Job 21 completed at 7.0s
Job 22 completed at 6.0s
Job 23 completed at 13.0s
Job 24 completed at 1.0s
Job 25 completed at 5.0s
Job 26 completed at 13.0s
Job 27 completed at 8.0s
Job 28 completed at 5.0s
Job 29 completed at 7.0s
Job 30 completed at 3.0s
Job 31 completed at 9.0s
Job 32 completed at 2.0s
Job 33 completed at 13.0s
Job 34 completed at 5.0s
Job 35 completed at 10.0s
Job 36 completed at 8.0s
Job 37 completed at 4.0s
Job 38 completed at 11.0s
Job 39 completed at 8.0s
Job 40 completed at 13.

# Mathematical Programming Algorithm (MPA)

The paper cites a decomposition algorithm (Branch-and-check) based on the previous formulation.

It is an exact formulation based on a master algorithm and a sequence algorithm:
- The master algorithm is responsible of assigning jobs to machines
- The sequence problem is responsible of finding the optimal sequence of jobs for each machine

In [22]:
s = MPA(execution_times = P_dict, setup_times = S_dict, t_max=10800) #  max_time = 1800
decision_variables,maximum_makespan = s.solve(options=options)
clear_output(wait=True)
for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

Job 0 scheduled in sequence of job 6 in machine 1
Job 6 scheduled in sequence of job 2 in machine 1
Job 12 scheduled in sequence of job 19 in machine 1
Job 16 scheduled in sequence of job 38 in machine 1
Job 38 scheduled in sequence of job 12 in machine 1
Job 10 scheduled in sequence of job 40 in machine 2
Job 18 scheduled in sequence of job 11 in machine 2
Job 23 scheduled in sequence of job 41 in machine 3
Job 28 scheduled in sequence of job 0 in machine 3
Job 50 scheduled in sequence of job 23 in machine 3
Job 26 scheduled in sequence of job 31 in machine 4
Job 22 scheduled in sequence of job 47 in machine 5
Job 5 scheduled in sequence of job 8 in machine 6
Job 34 scheduled in sequence of job 27 in machine 6
Job 0 scheduled in sequence of job 17 in machine 7
Job 33 scheduled in sequence of job 37 in machine 7
Job 0 scheduled in sequence of job 7 in machine 8
Job 1 scheduled in sequence of job 0 in machine 8
Job 39 scheduled in sequence of job 15 in machine 8
Job 0 scheduled in seque

# Fix-and-Optimize approach

In [23]:
greedy = Greedy(execution_times=P_dict, setup_times=S_dict)
solution = greedy.solve()

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

for j in s.N:
    print(f'Job {j} completed at {completion_times[j]}s')

Job 0 scheduled in sequence of job 43 in machine 1
Job 7 scheduled in sequence of job 12 in machine 1
Job 12 scheduled in sequence of job 15 in machine 1
Job 15 scheduled in sequence of job 29 in machine 1
Job 29 scheduled in sequence of job 0 in machine 1
Job 43 scheduled in sequence of job 7 in machine 1
Job 0 scheduled in sequence of job 8 in machine 2
Job 3 scheduled in sequence of job 0 in machine 2
Job 6 scheduled in sequence of job 3 in machine 2
Job 8 scheduled in sequence of job 45 in machine 2
Job 16 scheduled in sequence of job 22 in machine 2
Job 22 scheduled in sequence of job 6 in machine 2
Job 45 scheduled in sequence of job 16 in machine 2
Job 0 scheduled in sequence of job 5 in machine 3
Job 4 scheduled in sequence of job 46 in machine 3
Job 5 scheduled in sequence of job 20 in machine 3
Job 19 scheduled in sequence of job 4 in machine 3
Job 20 scheduled in sequence of job 19 in machine 3
Job 42 scheduled in sequence of job 0 in machine 3
Job 46 scheduled in sequence o

In [24]:
m = FixOpt(initial_solution=solution, setup_times=S_dict,execution_times=P_dict, N=N, M=M, N0=N0, 
           subproblem_size_adjust_rate=0.1, t_max = 3600, subproblem_runtime_limit=10, subproblem_size=5)

solution, makespan = m.solve()
clear_output(wait=True)
for i in M:
    for j in N0:
        for k in N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled after job {k} in machine {i}')
print(f'The makespan has been minimized to {makespan}s')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter Licens